In [234]:
# Scenarios: Metodo para generara escenarios
# Parametros:
# - source(string): Path del archivo de origen de informacion
# - separator(string): Delimitador de columnas
# - name(string): Nombre del escenario
# - unit(char): Medida, puede ser s: segundos, m: minutos, h: horas o d:dias
# - elapse(int): Cantidad de horas, minutos o seguntos transcurridos
# - quantity(int): Cantidad de medidas hacia atras a obtener
# - variable(int): Variable a medir

#BEGIN*****************************************************************************************************
import pandas as pd
import numpy as np


def Scenarios(source,separator,name, unit, elapse, quantity, variables):
    #Craga del dataframe data mediante lectura del archivo de origen
    data=pd.read_csv(source,sep=separator)
    
    dft = pd.DataFrame() 
    
    for i in range(0,len(variables)):
        
        variable=variables[i]
        #Sumatoria de la variable especificada agrupada por segundos
        def seconds(var):
            df1=data.groupby(["anio","mes","dia", "hora","minutos","segundos"],as_index=False) [var].sum()
            df2=df1[["anio","mes","dia", "hora","minutos","segundos",var]]
            df2[var]=df1.groupby(np.arange(len(df1)) // elapse)[var].cumsum()
            return df2.iloc[(elapse-1):(-1*elapse):elapse]
        #Sumatoria de la variable especificada agrupada por minutos
        def minutes(var):
            df1=data.groupby(["anio","mes","dia", "hora","minutos"],as_index=False) [var].sum()
            df2=df1[["anio","mes","dia", "hora","minutos",var]]
            df2[var]=df1.groupby(np.arange(len(df1)) // elapse)[var].cumsum()
            return df2.iloc[(elapse-1):(-1*elapse):elapse]
        #Sumatoria de la variable especificada agrupada por horas
        def hours(var):
            df1=data.groupby(["anio","mes","dia", "hora"],as_index=False) [var].sum()
            df2=df1[["anio","mes","dia", "hora",var]]
            df2[var]=df1.groupby(np.arange(len(df1)) // elapse)[var].cumsum()
            return df2.iloc[(elapse-1):(-1*elapse):elapse]
        #Sumatoria de la variable especificada agrupada por dias
        def days(var):
            df1=data.groupby(["anio","mes","dia"],as_index=False) [var].sum()
            df2=df1[["anio","mes","dia",var]]
            df2[var]=df1.groupby(np.arange(len(df1)) // elapse)[var].cumsum()
            return df2.iloc[(elapse-1):(-1*elapse):elapse]
        #Sumatoria de la variable especificada agrupada por meses
        def months(var):
            df1=data.groupby(["anio","mes"],as_index=False) [var].sum()
            df2=df1[["anio","mes",var]]
            df2[var]=df1.groupby(np.arange(len(df1)) // elapse)[var].cumsum()
            return df2.iloc[(elapse-1):(-1*elapse):elapse]
        #Sumatoria de la variable especificada agrupada por años
        def years(var):
            df1=data.groupby(["anio"],as_index=False) [var].sum()
            df2=df1[["anio",var]]
            df2[var]=df1.groupby(np.arange(len(df1)) // elapse)[var].cumsum()
            return df2.iloc[(elapse-1):(-1*elapse):elapse]
        #Diccionario de opciones de agrupamiento
        options = {"s":seconds, "m":minutes, "h":hours, "d":days, "M":months, "y":years}
        #Carga del dataframe df3 segun la opicon seleccionada del diccionario de opciones de agrupamiento
        df3 = options[unit](variable)
        #Generacion del primer campo de regrecion
        df3[variable+"-"+str(elapse)+unit] = df3[variable].shift()
        #Generacion del segundo al n(quantity) campo de regrecion
        for j in range(2,quantity):
            df3[variable+"-"+str(elapse*j)+unit] = df3[variable+"-"+str(elapse*(j-1))+unit].shift()
    
        if i==0:
            dft=df3
        else:
            frames=[dft,df3]
            dft=pd.merge(dft,df3)
    
    print(dft.head(5))
    #Escritura del archivo del escenario
    dft.to_csv("C:\\Users\\ADMIN\Documents\\Proyecto Modelo de Datos\\Scenarios\\"+name+".txt",sep=separator)
   
        

#END*****************************************************************************************************    


In [239]:
    
#Llamada a la funcion de generacion de escenarios
# Path del archivo de origen de informacion
path="C:\\Users\\ADMIN\Documents\\Proyecto Modelo de Datos\\HAMSTER.txt"
# Delimitador de columnas
sep="\t"
# Nombre del escenario
name="escenario_prueba"
# Unidad de medida, puede ser s: segundos, m: minutos, h: horas o d:dias
unit="h"
# Lapso de tiempo
elapse=1
# Cantidad de medidas hacia atras a obtener
quantity=4
# Variable a calcular, puede ser Sensor1 o Sensor2
var=("Sensor1",)
Scenarios(path,sep,name, unit, elapse, quantity,var)


   anio  mes  dia  hora  Sensor1  Sensor1-1h  Sensor1-2h  Sensor1-3h
0  2018    4   28    12        1         NaN         NaN         NaN
1  2018    4   28    13       12         1.0         NaN         NaN
2  2018    4   28    14       40        12.0         1.0         NaN
3  2018    4   28    15       10        40.0        12.0         1.0
4  2018    4   28    16        6        10.0        40.0        12.0
